In [1]:
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, f1_score, accuracy_score, precision_score, recall_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Define the path to the directory containing the feature files
directory_path = '/content/drive/MyDrive/FYP_dataset/features'

In [4]:
# Get a list of all feature file paths in the directory
feature_files = glob.glob(directory_path + '/*.csv')

In [5]:
print(len(feature_files))

1802


In [6]:
# Define lists to store the feature and target data
X = []
y_a = []
y_v = []

In [7]:
# Iterate over each feature file
for file in feature_files:
    # Read the feature file into a DataFrame
    df = pd.read_csv(file)

    # Extract the features and target variables
    features = df.drop(['Arousal_Value', 'Valence_Value', 'frameTime'], axis=1).values
    arousal = df['Arousal_Value'].values
    valence = df['Valence_Value'].values

    # Append the data to the lists
    X.append(features)
    y_a.append(arousal)
    y_v.append(valence)

In [8]:
# Concatenate the feature and target arrays
X = np.concatenate(X)
y_a = np.concatenate(y_a)
y_v = np.concatenate(y_v)


In [13]:
# Define the number of folds for cross-validation
num_folds = 5

# Initialize lists to store evaluation results
mse_a = []
mae_a = []
rmse_a = []
r2_a = []
f1_a = []
accuracy_a = []
precision_a = []

recall_a = []

In [14]:
%%time

# Perform tenfold cross-validation
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_a_train, y_a_test = y_a[train_index], y_a[test_index]


    # Initialize and train the SVR model
    lr_model_arosal = LinearRegression()

    lr_model_arosal.fit(X_train, y_a_train)

    # Evaluate the model on the test set
    predictions_a = lr_model_arosal.predict(X_test)


    # Calculate additional evaluation metrics
    mse_a.append(mean_squared_error(y_a_test, predictions_a))
    mae_a.append(mean_absolute_error(y_a_test, predictions_a))
    rmse_a.append(np.sqrt(mean_squared_error(y_a_test, predictions_a)))
    r2_a.append(r2_score(y_a_test, predictions_a))

    # Convert regression predictions to binary labels
    threshold = 0
    binary_predictions_a = (predictions_a >= threshold).astype(int)
    y_a_test_binary = (y_a_test >= threshold).astype(int)

    # Calculate F1-score, accuracy, precision, and recall for binary classification
    f1_a.append(f1_score(y_a_test_binary, binary_predictions_a))
    accuracy_a.append(accuracy_score(y_a_test_binary, binary_predictions_a))
    precision_a.append(precision_score(y_a_test_binary, binary_predictions_a))
    recall_a.append(recall_score(y_a_test_binary, binary_predictions_a))

CPU times: user 15.3 s, sys: 2.82 s, total: 18.1 s
Wall time: 13 s


In [15]:
# Calculate average performance across all folds
average_mse_a = np.mean(mse_a)
average_mae_a = np.mean(mae_a)
average_rmse_a = np.mean(rmse_a)
average_r2_a = np.mean(r2_a)
average_f1_a = np.mean(f1_a)
average_accuracy_a = np.mean(accuracy_a)
average_precision_a = np.mean(precision_a)
average_recall_a = np.mean(recall_a)

print(f'Average Arousal MSE: {average_mse_a:.4f}')
print(f'Average Arousal MAE: {average_mae_a:.4f}')
print(f'Average Arousal RMSE: {average_rmse_a:.4f}')
print(f'Average Arousal R2 Score: {average_r2_a:.4f}')
print(f'Average Arousal F1 Score: {average_f1_a:.4f}')
print(f'Average Arousal Accuracy: {average_accuracy_a:.4f}')
print(f'Average Arousal Precision: {average_precision_a:.4f}')
print(f'Average Arousal Recall: {average_recall_a:.4f}')

Average Arousal MSE: 0.0383
Average Arousal MAE: 0.1545
Average Arousal RMSE: 0.1956
Average Arousal R2 Score: 0.5292
Average Arousal F1 Score: 0.8496
Average Arousal Accuracy: 0.7919
Average Arousal Precision: 0.8142
Average Arousal Recall: 0.8883
